Use only the data at 2023


In [65]:
import pandas as pd
import numpy as np
import warnings

data_path = "data/"

In [66]:
df = pd.read_csv(f"{data_path}chicago_crime.csv")
df.shape[0]

8262433

In [67]:
df["Date"] = pd.to_datetime(df["Date"])
df_after = df[df["Date"].dt.year.isin([2021, 2022, 2023, 2024])]

In [ ]:
df_clean = df_after.copy()
df_clean = df_clean.sort_values(by="Date")
df_clean["Date"] = pd.to_datetime(df_clean["Date"], format="%Y-%m-%dT%H:%M:%S.%f")
df_clean["Year"] = df_clean["Date"].dt.year
df_clean["Month"] = df_clean["Date"].dt.month
df_clean["Day"] = df_clean["Date"].dt.day
df_clean["Hour"] = df_clean["Date"].dt.hour
df_clean["WeekDay"] = df_clean["Date"].dt.weekday
df_clean["IsWeekday"] = (df_clean["WeekDay"] < 5).astype(bool)
df_clean = df_clean.dropna(subset=["Longitude", "Latitude"])
df_clean["Location Description"] = df_clean["Location Description"].fillna(
    "OTHER (SPECIFY)"
)

,Column,Unique Values,Type,Is Null
0,ID,956325,int64,0
1,Case Number,956193,object,0
2,Date,467709,datetime64[ns],0
21,Location,287521,object,0
19,Latitude,287389,float64,0
20,Longitude,287303,float64,0
16,Y Coordinate,104195,float64,0
15,X Coordinate,64747,float64,0
3,Block,34899,object,0
18,Updated On,2430,object,0


In [69]:
location_mapping = {
    # Residential locations
    "RESIDENTIAL": [
        "APARTMENT",
        "RESIDENCE",
        "RESIDENCE - GARAGE",
        "RESIDENCE - PORCH / HALLWAY",
        "RESIDENCE - YARD (FRONT / BACK)",
        "HOUSE",
        "PORCH",
        "YARD",
        "GARAGE",
        "DRIVEWAY - RESIDENTIAL",
        "DRIVEWAY",
        "CHA APARTMENT",
        "CHA HALLWAY / STAIRWELL / ELEVATOR",
        "CHA HALLWAY",
        "CHA LOBBY",
        "CHA STAIRWELL",
        "CHA GROUNDS",
        "STAIRWELL",
        "BASEMENT",
        "PORCH",
        "HALLWAY",
        "VESTIBULE",
        "GANGWAY",
        "COLLEGE / UNIVERSITY - RESIDENCE HALL",
        "ROOF",
        "CHA ELEVATOR",  # Added
        "ELEVATOR",  # Added
    ],
    # Commercial retail
    "RETAIL": [
        "DEPARTMENT STORE",
        "SMALL RETAIL STORE",
        "GROCERY FOOD STORE",
        "CONVENIENCE STORE",
        "RETAIL STORE",
        "DRUG STORE",
        "APPLIANCE STORE",
        "PAWN SHOP",
        "AUTO / BOAT / RV DEALERSHIP",
        "LIQUOR STORE",
        "TAVERN / LIQUOR STORE",
        "CLEANING STORE",
        "NEWSSTAND",
    ],
    # Food and entertainment
    "FOOD_ENTERTAINMENT": [
        "RESTAURANT",
        "BAR OR TAVERN",
        "TAVERN",
        "MOVIE HOUSE / THEATER",
        "SPORTS ARENA / STADIUM",
        "BOWLING ALLEY",
        "POOL ROOM",
        "CASINO/GAMBLING ESTABLISHMENT",
        "ATHLETIC CLUB",
        "CLUB",  # Added
        "BANQUET HALL",  # Added
    ],
    # Transportation
    "TRANSPORTATION": [
        "CTA TRAIN",
        "CTA PLATFORM",
        "CTA BUS",
        "CTA STATION",
        "CTA BUS STOP",
        "CTA TRACKS - RIGHT OF WAY",
        'CTA "L" TRAIN',
        'CTA "L" PLATFORM',
        "CTA SUBWAY STATION",
        "CTA PROPERTY",
        "CTA PARKING LOT / GARAGE / OTHER PROPERTY",
        "TAXICAB",
        "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)",
        "VEHICLE NON-COMMERCIAL",
        "VEHICLE - COMMERCIAL",
        "VEHICLE - DELIVERY TRUCK",
        "VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS",
        "VEHICLE - COMMERCIAL: TROLLEY BUS",
        "OTHER COMMERCIAL TRANSPORTATION",
        "AUTO",
        "OTHER RAILROAD PROPERTY / TRAIN DEPOT",
        "RAILROAD PROPERTY",
        "TRUCK",  # Added
    ],
    # Street and outdoor public areas
    "STREET_OUTDOOR": [
        "STREET",
        "SIDEWALK",
        "ALLEY",
        "VACANT LOT / LAND",
        "VACANT LOT",
        "HIGHWAY / EXPRESSWAY",
        "PARKING LOT",
        "BRIDGE",
        "PARK PROPERTY",
        "LAKEFRONT / WATERFRONT / RIVERBANK",
        "BEACH",
        "FOREST PRESERVE",
        "FARM",
        "RIVER BANK",  # Added
    ],
    # Parking and vehicle related
    "PARKING": [
        "PARKING LOT / GARAGE (NON RESIDENTIAL)",
        "GAS STATION",
        "GAS STATION DRIVE/PROP.",
        "CAR WASH",
        "PARKING LOT",
        "CHA PARKING LOT / GROUNDS",
        "CHA PARKING LOT",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "AIRPORT PARKING LOT",
    ],
    # Financial institutions
    "FINANCIAL": [
        "BANK",
        "CURRENCY EXCHANGE",
        "ATM (AUTOMATIC TELLER MACHINE)",
        "CREDIT UNION",
        "SAVINGS AND LOAN",
    ],
    # Office and business
    "OFFICE_BUSINESS": [
        "COMMERCIAL / BUSINESS OFFICE",
        "MEDICAL / DENTAL OFFICE",
        "WAREHOUSE",
        "FACTORY / MANUFACTURING BUILDING",
        "OFFICE",
        "CONSTRUCTION SITE",
        "ANIMAL HOSPITAL",
    ],
    # Educational
    "EDUCATIONAL": [
        "SCHOOL - PUBLIC BUILDING",
        "SCHOOL - PUBLIC GROUNDS",
        "SCHOOL - PRIVATE BUILDING",
        "SCHOOL - PRIVATE GROUNDS",
        "COLLEGE / UNIVERSITY - GROUNDS",
        "LIBRARY",
        "SCHOOL YARD",
        "DAY CARE CENTER",
        "PUBLIC GRAMMAR SCHOOL",  # Added
    ],
    # Government and public services
    "GOVERNMENT": [
        "GOVERNMENT BUILDING / PROPERTY",
        "FEDERAL BUILDING",
        "HOSPITAL BUILDING / GROUNDS",
        "HOSPITAL",
        "FIRE STATION",
        "POLICE FACILITY / VEHICLE PARKING LOT",
        "NURSING / RETIREMENT HOME",
        "JAIL / LOCK-UP FACILITY",
        "POLICE FACILITY",  # Added
    ],
    # Religious
    "RELIGIOUS": ["CHURCH / SYNAGOGUE / PLACE OF WORSHIP"],
    # Airport
    "AIRPORT": [
        "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - SECURE AREA",
        "AIRPORT EXTERIOR - SECURE AREA",
        "AIRPORT EXTERIOR - NON-SECURE AREA",
        "AIRPORT TRANSPORTATION SYSTEM (ATS)",
        "AIRPORT VENDING ESTABLISHMENT",
        "AIRPORT/AIRCRAFT",
        "AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA",
        "AIRCRAFT",
    ],
    # Specialty retail/service
    "SPECIALTY_RETAIL": ["BARBERSHOP", "BARBER SHOP/BEAUTY SALON", "KENNEL"],
    # Water related
    "WATER": ["BOAT / WATERCRAFT"],
    # Abandoned/vacant
    "ABANDONED": ["ABANDONED BUILDING"],
    # Coin operated
    "COIN_OPERATED": ["COIN OPERATED MACHINE"],
    # Cemetery
    "CEMETERY": ["CEMETARY"],
    # Hotel
    "LODGING": ["HOTEL / MOTEL", "HOTEL", "MOTEL"],  # Added MOTEL
    "OTHER (SPECIFY)": ["OTHER"],
}
# Create reverse mapping (from specific location to group)
reverse_mapping = {}
for group, locations in location_mapping.items():
    for location in locations:
        reverse_mapping[location] = group


def map_to_location_group(location):
    if location in reverse_mapping:
        return reverse_mapping[location]
    if location != "OTHER (SPECIFY)":
        warnings.warn("Unknown location:" + location)
    return "OTHER"


df_clean["Location Group"] = df_clean["Location Description"].apply(
    map_to_location_group
)
df_clean["Location Group"] = df_clean["Location Description"].apply(
    map_to_location_group
)

In [74]:
df_analysis = pd.DataFrame(
    {
        "Column": df_clean.columns,
        "Unique Values": df_clean.nunique().values,
        "Type": df_clean.dtypes.values,
        "Is Null": df_clean.isnull().sum().values,
    }
)
df_analysis.sort_values("Unique Values", ascending=False)

,Column,Unique Values,Type,Is Null
0,ID,956325,int64,0
1,Case Number,956193,object,0
2,Date,467709,datetime64[ns],0
21,Location,287521,object,0
19,Latitude,287389,float64,0
20,Longitude,287303,float64,0
16,Y Coordinate,104195,float64,0
15,X Coordinate,64747,float64,0
3,Block,34899,object,0
18,Updated On,2430,object,0


In [71]:
# First create your year-based split
df_train = df_clean[df_clean["Year"].isin([2021, 2022, 2023])]
df_test = df_clean[df_clean["Year"] == 2024]

# Then sample from each to get your desired sizes
df_train = df_train.sample(n=200000, random_state=42)  # 200k training samples
df_test = df_test.sample(n=40000, random_state=42)  # 40k testing samples (0.2*200k)
impute_values = {
    "Ward": df_train["Ward"].mode()[0],
    "Community Area": df_train["Community Area"].mode()[0],
}
for column, value in impute_values.items():
    df_train[column] = df_train[column].fillna(value)
    df_test[column] = df_test[column].fillna(value)

In [72]:
df_train_sorted_lat = df_train["Latitude"].sort_values()
lat_bin_edges = np.linspace(
    df_train_sorted_lat.min(), df_train_sorted_lat.max(), 51
)  # 51 edges for 50 bins
df_train["lat_bin"] = pd.cut(
    df_train["Latitude"], bins=lat_bin_edges, include_lowest=True
)
df_test["lat_bin"] = pd.cut(
    df_test["Latitude"], bins=lat_bin_edges, include_lowest=True
)
df_train_sorted_lon = df_train["Longitude"].sort_values()
lon_bin_edges = np.linspace(
    df_train_sorted_lon.min(), df_train_sorted_lon.max(), 51
)  # 51 edges for 50 bins
df_train["lon_bin"] = pd.cut(
    df_train["Longitude"], bins=lon_bin_edges, include_lowest=True
)
df_test["lon_bin"] = pd.cut(
    df_test["Longitude"], bins=lon_bin_edges, include_lowest=True
)

In [75]:
# Create time of day and season features
hour_bins = [0, 6, 12, 18, 24]
hour_labels = ["Night", "Morning", "Afternoon", "Evening"]
season_bins = [0, 3, 6, 9, 12]
season_labels = ["Winter", "Spring", "Summer", "Fall"]

df_train["TimeCategory"] = pd.cut(
    df_train["Hour"], bins=hour_bins, labels=hour_labels, include_lowest=True
)
df_test["TimeCategory"] = pd.cut(
    df_test["Hour"], bins=hour_bins, labels=hour_labels, include_lowest=True
)
df_train["Season"] = pd.cut(
    df_train["Month"], bins=season_bins, labels=season_labels, include_lowest=True
)
df_test["Season"] = pd.cut(
    df_test["Month"], bins=season_bins, labels=season_labels, include_lowest=True
)

In [76]:
location_counts = df_train["Location Description"].value_counts()
threshold = len(df_train) * 0.01  # Keep categories that appear in at least 1% of data
keep_locations = location_counts[location_counts >= threshold].index


def map_rare_categories(df, column, keep_values):
    df_copy = df.copy()
    df_copy.loc[~df_copy[column].isin(keep_values), column] = "Other"
    return df_copy


df_train = map_rare_categories(df_train, "Location Description", keep_locations)
df_test = map_rare_categories(df_test, "Location Description", keep_locations)
df_train.to_csv(f"{data_path}processed_train_data.csv", index=False)
df_test.to_csv(f"{data_path}processed_test_data.csv", index=False)

In [77]:
df_train.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Month,Day,Hour,WeekDay,IsWeekday,Location Group,lat_bin,lon_bin,TimeCategory,Season
102884,13320746,JG556216,2023-12-09 12:30:00,0000X S WABASH AVE,0870,THEFT,POCKET-PICKING,RESTAURANT,False,False,...,12,9,12,5,False,FOOD_ENTERTAINMENT,"(41.879, 41.886]","(-87.632, -87.624]",Morning,Fall
7854131,13012380,JG187907,2023-03-16 14:15:00,005XX W ROOSEVELT RD,0460,BATTERY,SIMPLE,RESTAURANT,False,False,...,3,16,14,3,True,FOOD_ENTERTAINMENT,"(41.864, 41.871]","(-87.641, -87.632]",Afternoon,Winter
7703003,12707119,JF252064,2022-05-22 23:00:00,040XX E 134TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,5,22,23,6,False,RESIDENTIAL,"(41.644000000000005, 41.652]","(-87.533, -87.525]",Evening,Spring
7566496,12471702,JE360404,2021-08-16 12:15:00,027XX N CLYBOURN AVE,0860,THEFT,RETAIL THEFT,SMALL RETAIL STORE,False,False,...,8,16,12,0,True,RETAIL,"(41.924, 41.932]","(-87.682, -87.674]",Morning,Summer
7709702,12805196,JF369800,2022-08-21 21:00:00,022XX N HARLEM AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,...,8,21,21,6,False,RESIDENTIAL,"(41.917, 41.924]","(-87.807, -87.799]",Evening,Summer


In [78]:
df_test.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Month,Day,Hour,WeekDay,IsWeekday,Location Group,lat_bin,lon_bin,TimeCategory,Season
8201341,13639189,JH478419,2024-10-22 14:22:00,034XX S WESTERN AVE,0560,ASSAULT,SIMPLE,RESIDENCE,False,False,...,10,22,14,1,True,RESIDENTIAL,"(41.826, 41.834]","(-87.691, -87.682]",Afternoon,Fall
8228164,13673377,JH519827,2024-11-24 13:50:00,025XX S PRAIRIE AVE,0460,BATTERY,SIMPLE,Other,False,False,...,11,24,13,6,False,GOVERNMENT,"(41.841, 41.849]","(-87.624, -87.616]",Afternoon,Fall
8116614,13500008,JH311649,2024-06-18 17:15:00,006XX N SPRINGFIELD AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,6,18,17,1,True,RESIDENTIAL,"(41.886, 41.894]","(-87.724, -87.716]",Afternoon,Spring
8215565,13656266,JH499234,2024-11-07 12:00:00,014XX S MILLARD AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,11,7,12,3,True,STREET_OUTDOOR,"(41.856, 41.864]","(-87.724, -87.716]",Morning,Fall
8019385,13360837,JH142992,2024-01-30 00:01:00,007XX E 90TH ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,True,True,...,1,30,0,1,True,RESIDENTIAL,"(41.728, 41.735]","(-87.608, -87.599]",Night,Winter


In [79]:
df_train.shape, df_test.shape

((200000, 32), (40000, 32))